In [1]:
import matplotlib.pyplot as plt
import numpy as np

def cl_cd(alpha=False, cl_max=False, cl=False):
	# NACA 2412
	alpha_ref = np.array([-5, 0, 5, 10, 15, 20])
	reference_cl = np.array([-0.3, 0.25, 0.8, 1.25, 1.45, 1.3])
	reference_cd = np.array([0.01, 0.008, 0.01, 0.02, 0.05, 0.13])

	if cl_max:
		cl = np.max(reference_cl)
		cd = reference_cd[cl==reference_cl]
	elif cl is not False:
		cd = np.interp(cl, reference_cl, reference_cd)
		return cd
	elif alpha is not False:
		cl = np.interp(alpha, alpha_ref, reference_cl)
		cd = np.interp(alpha, alpha_ref, reference_cd)
	else:
		print("Wrong input for Cl_Cd, returning dummy values")
		return np.mean(reference_cl), np.mean(reference_cd)

	return cl, cd

def pt_mass(p_max):
	# Ref engine
	V = 12
	A = 60
	P = V*A
	W = 0.120
	W_P = W/P

	return W_P*p_max

def s_mass(s_wet):
	# Weight per surface area, guessed 1.3 kg/m^2
	return s_wet*1.3

def PT_T(P, V):
	return P*PT_eta(P,V)/V

def PT_eta(P, V):
	# Power train efficiency, fixed at 85% until better function
	return 0.85

def Calc_score(Performance):
	Weighting_factors = np.array([-4, 7, 25, 4, -4, 5, -3, 8, -12, 10, 18])
	scores = Weighting_factors*Performance

	return [np.sum(scores), scores]

In [21]:
MTOW = 15
BATT_WEIGHT = 3

# CONSTANTS
AIR_DEN = 1.25
GRAVITY = 9.82

# Energy storage in Wh, assuming 260 Wh/kg and 3 kg of batteries
bat_capacity = 260*3

max_thrust = 50
power = 600


# Needs a better estimation of Cd_fuse!!! Scaled with fuselage surface area
fusilage_cd = 0.02

#----- Optim
cruise_airspeed = 15 # m/s
max_power = 600 # Watt
wing_area = 2 # m^2
fusilage_area = 1 # m^2


In [39]:
max_cl, takeoff_cd = cl_cd(cl_max=True)
stall_velocity = (2*MTOW*GRAVITY/(max_cl*AIR_DEN*wing_area))**0.5

#takeoff_drag = 0.5*takeoff_cd*AIR_DEN*(fusilage_area + wing_area)*takeoff_velocity**2
#takeoff_accel = (max_thrust - takeoff_drag)/MTOW

dt = 0.0001
current_velocity = 0
current_time = 0
current_energy = 0
current_distance = 0

takeoff_accel = np.array(())
takeoff_velocity = np.array(())
x = np.array(())

while current_velocity < stall_velocity:
    current_drag = 0.5*takeoff_cd*AIR_DEN*(fusilage_area + wing_area)*current_velocity**2
    current_accel = (max_thrust - current_drag)/MTOW
    
    current_distance = current_distance + current_velocity*dt + 0.5*current_accel*dt**2
    current_velocity = current_velocity + current_accel*dt
    
    current_energy = current_energy + power*dt
    
    current_time = current_time+dt
    
    #takeoff_accel = np.append(takeoff_accel, current_accel)
    #takeoff_velocity = np.append(takeoff_velocity, current_velocity)
    #x = np.append(x, current_time)
    
takeoff_time = current_time
takeoff_energy = current_energy/3600
takeoff_distance = current_distance

print(f"Takeoff time: {takeoff_time} s")
print(f"Energy: {takeoff_energy} Wh")
print(f"Takeoff distance: {takeoff_distance} m")


Takeoff time: 2.8560000000016026 s
Energy: 0.47599999999970144 Wh
Takeoff distance: [13.22615323] m


In [ ]:

drag = 0.5*cruise_cd*AIR_DEN*(fusilage_area + wing_area)*cruise_airspeed**2
lift = 0.5*cruise_cl*AIR_DEN*(wing_area)*cruise_airspeed**2